In [1]:
import os, sys, email
import numpy as np
import pandas as pd
from nltk import tokenize
import pprint, re, time

pd.options.display.max_colwidth = 1000

In [2]:
sys.path.append(os.path.abspath(".."))
from loadFile import getFileList, getDirList, fileToDataFrame

# ディレクトリ 内のメールファイルを読み込む
directory_path = "/Users/taroaso/myprojects/OpenIE/wiki-research-l/2020-July"
file_list = getFileList(directory_path)

# テキストファイルをデータフレームに格納する
mail_df, thread_df = fileToDataFrame(file_list)

In [31]:
s_list = []
for i,content in mail_df['Content'].items():
    cleaned_lines = []
    original_lines = content.splitlines()
    for line in original_lines:
        if line == '':
            continue 
        elif len(line.split()) < 5:
            continue
        elif re.match(r'\W{2,}.+',line):
            continue
        elif re.match(r'_{2,}.+',line):
            continue
        elif re.match(r'^(On \w{3}, \w{3} [0-9]{1,2}, [0-9]{4} at [0-9]{1,2}:[0-9]{1,2} (A|P)M .+)',line) is None:
            cleaned_lines.append(line)
        else:
            break
    cleaned_message = ' '.join(map(str, cleaned_lines))
    s_list.append(cleaned_message)
mail_df['Cleaned_Content'] = s_list

In [29]:
for line in mail_df['Content'][8].splitlines():
    if line == '':
        print('NULL')
        continue 
    elif len(line.split()) < 5:
        print('3token:{}'.format(line))
        continue
    elif re.match(r'\W{2,}.+',line):
        print('KIGO2:{}'.format(line))
        continue
    elif re.match(r'_{2,}.+',line):
        print('2__:{}'.format(line))
        continue
    elif re.match(r'^(On \w{3}, \w{3} [0-9]{1,2}, [0-9]{4} at [0-9]{1,2}:[0-9]{1,2} (A|P)M .+)',line) is None:
        print('CORRECT:{}'.format(line))
    else:
        break

KIGO2:*** Apologies for cross-posting ***
NULL
NULL
CORRECT:COLING 2020 Third Workshop on NLP for Internet Freedom  (NLP4IF):
3token:Censorship, Disinformation, and Propaganda
NULL
NULL
NULL
3token:First Call for Papers
NULL
NULL
NULL
3token:Workshop website: http://www.netcopia.net/nlp4if/
NULL
CORRECT:Co-located with COLING 2020, Barcelona, Spain
NULL
CORRECT:Accepted papers will be published in the COLING 2020 NLP4IF Workshop
3token:Proceedings
NULL
NULL
3token: Important Dates
NULL
KIGO2:* Submission deadline: September 1, 2020 (23:59 PM Pacific Standard Time)
NULL
KIGO2:* Notification of acceptance: October 1, 2020
NULL
KIGO2:* Camera-ready papers due: October 20, 2020
NULL
KIGO2:* Workshop: December 12, 2020
NULL
NULL
CORRECT:NLP4IF (http://www.netcopia.net/nlp4if/) is dedicated to NLP methods that
CORRECT:potentially contribute or hamper  the free flow of information on the
CORRECT:Internet, their impact, and to our understanding of the issues that arise
CORRECT:in this area. We

In [32]:
mail_df

,file_path,Message-ID,From,Date,Subject,Content,Cleaned_Content
0,/Users/taroaso/myprojects/OpenIE/wiki-research-l/2020-July/2020-July-30,<CAHTOhHs0k6q89WAHek45sA9ePjLqYhYEEq0pu6GOdX1pqBL1Tg@mail.gmail.com>,haifengz.suny at gmail.com (Haifeng Zhang),"Wed, 22 Jul 2020 10:23:48 -0400",[Wiki-research-l] Qid to Page ID,"Paul,\nSorry, I did not read your response carefully.\nWill try the wikimapper API as suggested.\n\nThanks a lot!\n\nOn Tue, Jul 21, 2020 at 1:05 PM Paul Albert <mrpaulalbert at gmail.com> wrote:\n\n>\n> I've been using the Wikimapper API mentioned here:\n>\n>\n> https://opendata.stackexchange.com/questions/5467/mapping-all-wikipedia-urls-to-wikidata-entity-ids\n>\n>\n> Regards,\n>\n> Paul Albert\n>\n> ﻿On 7/21/20, 12:39 PM, ""Wiki-research-l on behalf of Haifeng Zhang"" <\n> wiki-research-l-bounces at lists.wikimedia.org on behalf of\n> haifengz.suny at gmail.com> wrote:\n>\n> Hi all,\n> Are you aware of any API that maps wikidata QIDs to their article Page\n> IDs?\n>\n> Thanks,\n> Haifeng\n> _______________________________________________\n> Wiki-research-l mailing list\n> Wiki-research-l at lists.wikimedia.org\n> https://lists.wikimedia.org/mailman/listinfo/wiki-research-l\n> _______________________________________________\n> Wiki-research-l mailing...","Sorry, I did not read your response carefully. Will try the wikimapper API as suggested."
1,/Users/taroaso/myprojects/OpenIE/wiki-research-l/2020-July/2020-July-08,<CAJXKj+owEP_UyX2A17x+4NrsB76zv9puwpbPE8ZPBpfD0CRARQ@mail.gmail.com>,jlayton at wikimedia.org (Janna Layton),"Thu, 9 Jul 2020 12:26:49 -0700","[Wiki-research-l] [Wikimedia Research Showcase] July 15,\n\t2020: Medical Knowledge on Wikipedia","Hi all,\n\nThe next Research Showcase will be live-streamed on Wednesday, July 15, at\n9:30 AM PDT/16:30 UTC.\n\nWikipedia is one of the most important online resources for health\ninformation. This has been especially highlighted during the Covid-19\npandemic: since the beginning of the year more than 5000 articles related\nto Covid-19 have been created receiving more than 400M pageviews.\nTherefore, for this month’s showcase our two invited speakers will help us\nget a better understanding of the state of medical knowledge in Wikipedia.\nIn the first talk, Denise Smith will give an overview on how Wikipedia's\nhealth content is used by different audiences (public, students, or\npractitioners). In the second talk, Giovanni Colavizza will present results\non how editors on Wikipedia find, select, and integrate scientific\ninformation on Covid-19 into Wikipedia articles.\n\nYouTube stream: https://www.youtube.com/watch?v=qIV26lWrD9c\n\nAs usual, you can join the conversation on IRC ...","The next Research Showcase will be live-streamed on Wednesday, July 15, at Wikipedia is one of the most important online resources for health information. This has been especially highlighted during the Covid-19 pandemic: since the beginning of the year more than 5000 articles related to Covid-19 have been created receiving more than 400M pageviews. Therefore, for this month’s showcase our two invited speakers will help us get a better understanding of the state of medical knowledge in Wikipedia. In the first talk, Denise Smith will give an overview on how Wikipedia's health content is used by different audiences (public, students, or practitioners). In the second talk, Giovanni Colavizza will present results on how editors on Wikipedia find, select, and integrate scientific information on Covid-19 into Wikipedia articles. As usual, you can join the conversation on IRC at #wikimedia-research. You can also watch our past research showcases here: Wikipedia for health information - Si..."
2,/Users/taroaso/myprojects/OpenIE/wiki-research-l/2020-July/2020-July-01,<CAE4fJj_b7k8yeqmz19a-seo3gtL9Mg1nvQO0oPUtrxvCFMkEaw@mail.gmail.com>,ziningy1 at andrew.cmu.edu (Ethan Ye),"Wed, 1 Jul 2020 17:54:24 -0400",[Wiki-research-l] Wikipedia ORES Research Study Looking for\n\tParticipants,"Hi Stella,

In [25]:
from preProcess import cleaningText
s_list = []
for i,content in mail_df['Content'].items():
    sentences = cleaningText(content)
    s_list.append(sentences)
mail_df['Cleaned_Content'] = s_list

In [26]:
mail_df

,file_path,Message-ID,From,Date,Subject,Content,Cleaned_Content
0,/Users/taroaso/myprojects/OpenIE/wiki-research-l/2020-July/2020-July-30,<CAHTOhHs0k6q89WAHek45sA9ePjLqYhYEEq0pu6GOdX1pqBL1Tg@mail.gmail.com>,haifengz.suny at gmail.com (Haifeng Zhang),"Wed, 22 Jul 2020 10:23:48 -0400",[Wiki-research-l] Qid to Page ID,"Paul,\nSorry, I did not read your response carefully.\nWill try the wikimapper API as suggested.\n\nThanks a lot!\n\nOn Tue, Jul 21, 2020 at 1:05 PM Paul Albert <mrpaulalbert at gmail.com> wrote:\n\n>\n> I've been using the Wikimapper API mentioned here:\n>\n>\n> https://opendata.stackexchange.com/questions/5467/mapping-all-wikipedia-urls-to-wikidata-entity-ids\n>\n>\n> Regards,\n>\n> Paul Albert\n>\n> ﻿On 7/21/20, 12:39 PM, ""Wiki-research-l on behalf of Haifeng Zhang"" <\n> wiki-research-l-bounces at lists.wikimedia.org on behalf of\n> haifengz.suny at gmail.com> wrote:\n>\n> Hi all,\n> Are you aware of any API that maps wikidata QIDs to their article Page\n> IDs?\n>\n> Thanks,\n> Haifeng\n> _______________________________________________\n> Wiki-research-l mailing list\n> Wiki-research-l at lists.wikimedia.org\n> https://lists.wikimedia.org/mailman/listinfo/wiki-research-l\n> _______________________________________________\n> Wiki-research-l mailing...","Paul, Sorry, I did not read your response carefully. Will try the wikimapper API as suggested. Thanks a lot!"
1,/Users/taroaso/myprojects/OpenIE/wiki-research-l/2020-July/2020-July-08,<CAJXKj+owEP_UyX2A17x+4NrsB76zv9puwpbPE8ZPBpfD0CRARQ@mail.gmail.com>,jlayton at wikimedia.org (Janna Layton),"Thu, 9 Jul 2020 12:26:49 -0700","[Wiki-research-l] [Wikimedia Research Showcase] July 15,\n\t2020: Medical Knowledge on Wikipedia","Hi all,\n\nThe next Research Showcase will be live-streamed on Wednesday, July 15, at\n9:30 AM PDT/16:30 UTC.\n\nWikipedia is one of the most important online resources for health\ninformation. This has been especially highlighted during the Covid-19\npandemic: since the beginning of the year more than 5000 articles related\nto Covid-19 have been created receiving more than 400M pageviews.\nTherefore, for this month’s showcase our two invited speakers will help us\nget a better understanding of the state of medical knowledge in Wikipedia.\nIn the first talk, Denise Smith will give an overview on how Wikipedia's\nhealth content is used by different audiences (public, students, or\npractitioners). In the second talk, Giovanni Colavizza will present results\non how editors on Wikipedia find, select, and integrate scientific\ninformation on Covid-19 into Wikipedia articles.\n\nYouTube stream: https://www.youtube.com/watch?v=qIV26lWrD9c\n\nAs usual, you can join the conversation on IRC ...","Hi all, The next Research Showcase will be live-streamed on Wednesday, July 15, at 9:30 AM PDT/16:30 UTC. Wikipedia is one of the most important online resources for health information. This has been especially highlighted during the Covid-19 pandemic: since the beginning of the year more than 5000 articles related to Covid-19 have been created receiving more than 400M pageviews. Therefore, for this month’s showcase our two invited speakers will help us get a better understanding of the state of medical knowledge in Wikipedia. In the first talk, Denise Smith will give an overview on how Wikipedia's health content is used by different audiences (public, students, or practitioners). In the second talk, Giovanni Colavizza will present results on how editors on Wikipedia find, select, and integrate scientific information on Covid-19 into Wikipedia articles. YouTube stream: https://www.youtube.com/watch?v=qIV26lWrD9c As usual, you can join the conversation on IRC at #wikimedia-research...."
2,/Users/taroaso/myprojects/OpenIE/wiki-research-l/2020-July/2020-July-01,<CAE4fJj_b7k8yeqmz19a-seo3gtL9Mg1nvQO0oPUtrxvCFMkEaw@mail.gmail.com>,ziningy1 at andrew.cmu.edu (Ethan Ye),"Wed, 1 Jul 2020 17:54:24 -0400",[Wiki-research-l] Wikipedia ORES Research Study Looking for\n\tPart

In [15]:
#　センテンスのリストを作る
sentence_list = []
for content in mail_df['Cleaned_Content']:
    sentences = tokenize.sent_tokenize(content)
    for i,s in enumerate(sentences):
        sentences[i] = re.sub(r'\n{1,}',' ',s)
        sentences[i] = re.sub(r'\s{2,}',' ',sentences[i])
        sentences[i] = re.sub(r'\W{2,}',' ',sentences[i])
        sentences[i] = re.sub(r'_{2,}',' ',sentences[i])
    for sentence in sentences:
        sentence_list.append(sentence)
sentence_list

ul 2020 at 06:43 Mackenzie Lemieux mackenzie.lemieux at gmail.com wrote Dear Wiki Community My name is Mackenzie Lemieux and I am a neuroscience researcher at the Salk Institute for Biological Studies and I am interested in exploring biases on Wikipedia.',
 ' My research hypothesis is that gender or ethnicity mediate the rate of flagging and deletion of pages for women in STEM.',
 "I hope to retrospectively analyze Wikipedia's deletion history harvest the biographical articles about scientists that have been created over the past n years and then confirm the gender and ethnicity of a large sample.",
 " It appears that we can identify deleted pages with Wikipedia's deletion log https en.wikipedia.org/wiki/Wikipedia:Deletion_log but to actually see the page that was deleted we need to be members of one of these Wikipedia user groups Administrators https en.wikipedia.org/wiki/Wikipedia:Administrators Oversighters https en.wikipedia.org/wiki/Wikipedia:Oversight Researchers https en.wikiped

In [17]:
sent_length = {}
for i, sentence in enumerate(sentence_list):
    sent_length[i] = len(sentence.split())
print(sent_length)

{0: 9, 1: 7, 2: 3, 3: 18, 4: 12, 5: 30, 6: 24, 7: 24, 8: 25, 9: 15, 10: 68, 11: 14, 12: 19, 13: 31, 14: 1, 15: 2, 16: 13, 17: 2, 18: 26, 19: 49, 20: 10, 21: 24, 22: 11, 23: 14, 24: 15, 25: 21, 26: 40, 27: 3, 28: 4, 29: 1, 30: 24, 31: 5, 32: 18, 33: 2, 34: 30, 35: 21, 36: 34, 37: 47, 38: 25, 39: 17, 40: 10, 41: 18, 42: 9, 43: 6, 44: 21, 45: 13, 46: 17, 47: 2, 48: 23, 49: 23, 50: 7, 51: 43, 52: 16, 53: 10, 54: 49, 55: 13, 56: 23, 57: 3, 58: 7, 59: 20, 60: 3, 61: 6, 62: 10, 63: 13, 64: 5, 65: 10, 66: 13, 67: 26, 68: 27, 69: 25, 70: 4, 71: 34, 72: 106, 73: 34, 74: 24, 75: 10, 76: 51, 77: 35, 78: 120, 79: 51, 80: 1, 81: 17, 82: 13, 83: 24, 84: 54, 85: 3, 86: 3, 87: 35, 88: 13, 89: 17, 90: 21, 91: 25, 92: 15, 93: 16, 94: 16, 95: 19, 96: 1, 97: 20, 98: 6, 99: 26, 100: 13, 101: 2, 102: 4, 103: 9, 104: 47, 105: 27, 106: 68, 107: 24, 108: 53, 109: 46, 110: 35, 111: 23, 112: 51, 113: 14, 114: 24, 115: 17, 116: 18, 117: 56, 118: 36, 119: 13, 120: 24, 121: 54, 122: 3, 123: 84, 124: 11, 125: 78, 126

In [18]:
sentence_list[156]

' Please fill this form to register for CICM 2020 https forms.gle/oS5BVGDf6LgDGDiK8 SCIENTIFIC PROGRAM The program of the conference is available under https easychair.org/smart-program/CICM-13 all times are in CEST timezone UTC+2 INVITED SPEAKERS Kevin Buzzard Imperial College London UK Formalizing Undergraduate Mathematics Catherine Dubois ENSIIE CNRS Evry France Formally Verified Constraints Solvers a Guided Tour Christian Szegedy Google Research Mountain View CA USA A Promising Path Towards Autoformalization and General Artificial Intelligence INVITED WORKSHOP SPEAKERS Freek Wiedijk Radboud University Nijmegen NL Formal Proof for the Future Fairouz Kamareddine Heriot-Watt University UK TBA AFFILIATED WORKSHOPS AND DOCTORAL PROGRAMME NFM 2020 Workshop on Natural Formal Mathematics https cicm-conference.org/2020/cicm.php?event=NFM Doctoral Programme https cicm-conference.org/2020/cicm.php?event=doctoral Subject Digest Footer   Wiki-research-l mailing list Wiki-research-l at lists.wik

In [25]:
id_length = sorted(sent_length.items(), key=lambda x:x[1], reverse=True)
sent_cnt_list = []
for tpl in id_length:
    sent_cnt_list.append([tpl[0],tpl[1],sentence_list[tpl[0]]])

In [29]:
sent_cnt_df = pd.DataFrame(sent_cnt_list, columns=('idx','cnt','sentence'))
sent_cnt_df.to_csv('analysis/output/sent_cnt_df.csv')